In [139]:
import time
import os 
from dotenv import load_dotenv

import spotipy
from spotipy.oauth2 import SpotifyOAuth

import pprint

import pandas as pd

In [34]:
def get_spotify_client(client_id, client_secret, redirect_uri, scope):
    return spotipy.Spotify(auth_manager=SpotifyOAuth(
        client_id=client_id, 
        client_secret=client_secret, 
        scope=scope,
        redirect_uri=redirect_uri
    ))

In [35]:
load_dotenv()
client_id = os.environ['CLIENT_ID']
client_secret = os.environ['CLIENT_SECRET']
redirect_uri = 'http://localhost:3000'
scope = "user-library-read user-top-read"

sp = get_spotify_client(client_id, client_secret, redirect_uri, scope)

In [89]:
def get_top_stats(client, limit=20, offset=0, time_range='medium_term', top_type='artists'):
    if top_type == 'artists':
        return client.current_user_top_artists(limit, offset, time_range)
    elif top_type == 'tracks':
        return client.current_user_top_tracks(limit, offset, time_range)

In [163]:
def get_top_artists(client, limit=90, time_range='long_term'):
    if limit <= 50:
        items = get_top_stats(client, limit=limit, time_range=time_range, top_type='artists')['items'] 
    else:
        items = get_top_stats(client, limit=40, time_range=time_range, top_type='artists')['items'] 
        items += get_top_stats(client, limit=limit-40, offset=40, time_range=time_range, top_type='artists')['items'] 
    return pd.DataFrame(items, columns=['name','genres','popularity'])

In [168]:
get_top_artists(sp, 20, 'short_term')

,name,genres,popularity
0,Radiohead,"[alternative rock, art rock, melancholia, oxfo...",78
1,My Chemical Romance,"[emo, modern rock, pop punk, pov: indie, rock]",75
2,Pixies,"[alternative rock, boston rock, permanent wave...",68
3,Weezer,"[alternative rock, modern power pop, modern ro...",71
4,Korn,"[alternative metal, funk metal, hard rock, nu ...",73
5,Have A Nice Life,"[blackgaze, doomgaze, dream pop, nu gaze]",52
6,Mitski,"[brooklyn indie, pov: indie]",76
7,Slipknot,"[alternative metal, nu metal, rap metal, rock]",76
8,The Clash,"[alternative rock, permanent wave, punk, rock]",68
9,System Of A Down,"[alternative metal, nu metal, rap metal, rock]",77


In [166]:
artists_df = get_top_artists(sp, 90, 'medium_term')

In [167]:
artists_df.to_csv('top90artists_med.csv')